# NMEA Reader - National Marine Electronics Association

Created by Michael George (AKA Logiqx)

This module was created because the NMEAReader of pynmeagps was way too slow for my liking.

NMEAReader took around 16 seconds for my test file, whereas populating numpy arrays with this module took about 2 seconds.

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import io
import time

from datetime import datetime, timezone

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class NmeaReader(BaseReader):
    '''NMEA file - National Marine Electronics Association'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self, ignoreChecksums=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.readlines()
                       
            if len(self.buffer) > 0:
                self.readData(ignoreChecksums=ignoreChecksums)


    def readData(self, ignoreChecksums=False):
        '''Read GGA and RMC messages into memory'''

        # Lean buffer will contain combined messages
        leanBuffer = bytearray()

        hhmmss = None
        rmcPayload = ggaPayload = None

        for sentence in self.buffer:
            fields = self.readSentence(sentence, ignoreChecksums=ignoreChecksums)
            typeCode = fields[0]

            if typeCode == b'GGA' or typeCode == b'RMC':
                if fields[1] != hhmmss:
                    # GGA and RMC both have to be present with the same timestamp
                    if ggaPayload and rmcPayload:
                        # Ignore messages without longitude and latitude values
                        if rmcPayload[3] and rmcPayload[5]:
                            leanBuffer += b','.join(ggaPayload[6:15] + rmcPayload[1:16])
                            leanBuffer += b'\n'
                            self.numRecords += 1

                    rmcPayload = None
                    ggaPayload = None

                hhmmss = fields[1]

                if typeCode == b'RMC':
                    rmcPayload = fields
                    numRmcFields = len(rmcPayload)
                else:
                    ggaPayload = fields

        if rmcPayload and ggaPayload:
            leanBuffer += b','.join(ggaPayload[6:15] + rmcPayload[1:16])
            leanBuffer += b'\n'
            self.numRecords += 1
        
        if self.numRecords > 0:
            dtype = self.getDatatype(numRmcFields)
            self.rawData = np.genfromtxt(io.BytesIO(leanBuffer), dtype=dtype, delimiter=",")
            self.consumeRawData()


    def readSentence(self, sentence, ignoreChecksums=False):
        '''Read NMEA sentence and validate checksum'''

        # Check for $ symbol (ASCII 0x24)
        if sentence[0] != 0x24:
            raise ValueError('Sentence does not start with a dollar $ symbol')
            
        asteriskIdx = sentence.find(b'*')
        if asteriskIdx == -1:
            raise ValueError('Sentence does not contain an asterisk * symbol')
        
        # Evaluate checksum
        if ignoreChecksums == False:
            expectedChecksum = int(sentence[asteriskIdx + 1: asteriskIdx + 3], 16)

            actualChecksum = 0
            for byte in sentence[1:asteriskIdx]:
                actualChecksum ^= byte

            if actualChecksum != expectedChecksum:
                raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        fields = sentence[3:asteriskIdx].split(b',')

        return fields


    def getDatatype(self, numRmcFields):
        '''Get the datatype of the combined GGA and RMC payload'''

        # Standard GGA payload excluding hhmmss, lat, ns, lon, ew
        dtype = [
            ('quality', 'u1'),    # GPS Quality Indicator (non null)
            ('numSv', 'u1'),      # Number of satellites in use, 00-12
            ('hdop', 'f4'),       # Horizontal Dilution of precision (meters)
            ('alt', 'f8'),        # Antenna Altitude above/below mean-sea-level (geoid) (in meters)
            ('altUnit', 'U1'),    # Units of antenna altitude, meters
            ('geoSep', 'f4'),     # Geoidal separation, the difference between the WGS-84 earth ellipsoid and mean-sea-level
            ('geoSepUnit', 'U1'), # Units of geoidal separation, meters
            ('dgpsAge', 'u2'),    # Age of differential GPS data, time in seconds since last SC104 type 1 or 9 update
            ('dgpsId', 'u2')      # Differential reference station ID, 0000-1023
        ]

        # Standard RMC payload
        dtype.extend([
            ('hhmmss', 'f8'),     # UTC of this position report as hhmmss.ss
            ('status', 'U1'),     # Status, A = Valid, V = Warning
            ('lat', 'f8'),        # Latitude as ddmm.mm - dd is degrees, mm.mm is minutes
            ('ns', 'U1'),         # N or S (North or South)
            ('lon', 'f8'),        # Longitude as ddmm.mm - dd is degrees, mm.mm is minutes
            ('ew', 'U1'),         # E or W (East or West)
            ('sog', 'f4'),        # Speed over ground, knots
            ('cog', 'f4'),        # Track made good, degrees true
            ('ddmmyy', 'u4'),     # Date, ddmmyy
            ('magVar', 'f4'),     # Magnetic Variation, degrees
            ('magVarEw', 'U1')    # E or W
        ])

        if numRmcFields >= 13:
            dtype.extend([
                # FAA mode indicator (NMEA 2.3 and later)
                # A=autonomous, D=differential, E=Estimated, M=Manual input, N=not valid, S=Simulator
                ('faaMode', 'U1')
            ])

        if numRmcFields >= 14:
            dtype.extend([
                # Nav Status (NMEA 4.1 and later)
                # A=autonomous, D=differential, E=Estimated, M=Manual input, N=not valid, S=Simulator, V=Valid
                ('navStatus', 'U1')
            ])

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert latitude and longitude from degrees + minutes to to degrees
        vfunc = np.vectorize(getDegrees)
        self.data['lat'] = vfunc(self.rawData['lat'], self.rawData['ns'])
        self.data['lon'] = vfunc(self.rawData['lon'], self.rawData['ew'])

        # Copy elevation
        self.data['ele'] = self.rawData['alt']

        # Convert SOG from knots to m/s, limiting to 3 decimal places
        self.data['sog'] = np.round(self.rawData['sog'] * 1852 / 3600, 3)

        # Copy COG
        self.data['cog'] = np.round(self.rawData['cog'], 3)

        # Convert ddmmyy + hhmmss to regular timestamp
        vfunc = np.vectorize(getDateTime)
        self.data['ts'] = vfunc(self.rawData['ddmmyy'], self.rawData['hhmmss'])

        # Copy satellites
        self.data['sat'] = self.rawData['numSv']

        # Copy HDOP
        self.data['hdop'] = self.rawData['hdop']
            

def getDegrees(ddmm, direction):
    '''Convert NMEA degrees and minutes to decimal degrees'''

    # Convert from degrees and minutes to decimal degrees, limiting to 7 decimal places
    degrees = np.round(np.floor(ddmm / 100) + ddmm % 100 / 60, 7)

    if direction == 'S' or direction == 'W':
        degrees = -degrees    

    return degrees


def getDateTime(ddmmyy, hhmmss):
    '''Decode date/time'''

    day = ddmmyy // 10000
    month = ddmmyy // 100 % 100

    year = ddmmyy % 100
    if year > 80:
        year += 1900
    else:
        year += 2000

    hhmmssi = int(hhmmss)

    hour = hhmmssi // 10000
    minute = hhmmssi // 100 % 100
    second = hhmmssi % 100
    microsecond = int(hhmmss * 1000) % 1000 * 1000

    return datetime(year, month, day, hour, minute, second, microsecond, tzinfo=timezone.utc).timestamp()

## Unit Tests

In [3]:
class TestGt31Data(unittest.TestCase):
    '''Class to GT-31 data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(gt31Reader.numRecords, 4161)

        for fieldName in gt31Reader.data:
            self.assertEqual(gt31Reader.data[fieldName].size, 4161)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(gt31Reader.data['hdop'].min(), np.float32(0.8))
        self.assertEqual(gt31Reader.data['hdop'].max(), np.float32(2.0))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(gt31Reader.data['sat'].min(), np.uint8(4))
        self.assertEqual(gt31Reader.data['sat'].max(), np.uint8(10))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(gt31Reader.data['ts'].min(), np.float64(1649672182.0))
        self.assertEqual(gt31Reader.data['ts'].max(), np.float64(1649678792.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(gt31Reader.data['lat'].min(), np.float64(50.5710167))
        self.assertEqual(gt31Reader.data['lat'].max(), np.float64(50.5833333))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(gt31Reader.data['lon'].min(), np.float64(-2.4620500))
        self.assertEqual(gt31Reader.data['lon'].max(), np.float64(-2.4563000))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(gt31Reader.data['ele'].min(), np.float64(-3.02))
        self.assertEqual(gt31Reader.data['ele'].max(), np.float64(11.93))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(gt31Reader.data['sog'].min(), np.float32(0.129))
        self.assertEqual(gt31Reader.data['sog'].max(), np.float32(16.827))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(gt31Reader.data['cog'].min(), np.float32(0.01))
        self.assertEqual(gt31Reader.data['cog'].max(), np.float32(359.92))

In [4]:
class TestFtechData(unittest.TestCase):
    '''Class to ftech data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(ftechReader.numRecords, 82409)

        for fieldName in ftechReader.data:
            self.assertEqual(ftechReader.data[fieldName].size, 82409)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(ftechReader.data['hdop'].min(), np.float32(0.73))
        self.assertEqual(ftechReader.data['hdop'].max(), np.float32(4.64))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(ftechReader.data['sat'].min(), np.uint8(3))
        self.assertEqual(ftechReader.data['sat'].max(), np.uint8(11))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(ftechReader.data['ts'].min(), np.float64(1375389686.279))
        self.assertEqual(ftechReader.data['ts'].max(), np.float64(1375398017.800))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(ftechReader.data['lat'].min(), np.float64(41.6302883))
        self.assertEqual(ftechReader.data['lat'].max(), np.float64(41.6360033))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(ftechReader.data['lon'].min(), np.float64(-70.2948667))
        self.assertEqual(ftechReader.data['lon'].max(), np.float64(-70.2740267))

    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(ftechReader.data['ele'].min(), np.float64(-116.7))
        self.assertEqual(ftechReader.data['ele'].max(), np.float64(118.2))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(gt31Reader.data['sog'].min(), np.float32(0.129))
        self.assertEqual(gt31Reader.data['sog'].max(), np.float32(16.827))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(gt31Reader.data['cog'].min(), np.float32(0.01))
        self.assertEqual(gt31Reader.data['cog'].max(), np.float32(359.92))

In [5]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.nmea')
    gt31Reader = NmeaReader(filename)

    filename = os.path.join(projdir, 'sessions', '20130801-ftech', 'GPSXX006_p1.nmea')
    ftechReader = NmeaReader(filename)

    pc1 = time.perf_counter()
    gt31Reader.load()
    ftechReader.load()
    pc2 = time.perf_counter()
    
    print("\nNMEA files loaded in in %0.2f seconds" % (pc2 - pc1))


NMEA files loaded in in 2.98 seconds


In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..................
----------------------------------------------------------------------
Ran 18 tests in 0.026s

OK
